1\. **PCA on 3D dataset**

* Generate a dataset with 3 features each with N entries (N being ${\cal O}(1000)$). With $N(\mu,\sigma)$ the normali distribution with mean $\mu$ and $\sigma$  standard deviation, generate the 3 variables $x_{1,2,3}$ such that:
    * $x_1$ is distributed as $N(0,1)$
    * $x_2$ is distributed as $x_1+N(0,3)$
    * $x_3$ is given by $2x_1+x_2$
* Find the eigenvectors and eigenvalues of the covariance matrix of the dataset
* Find the eigenvectors and eigenvalues using SVD. Check that the two procedures yield to same result
* What percent of the total dataset's variability is explained by the principal components? Given how the dataset was constructed, do these make sense? Reduce the dimensionality of the system so that at least 99% of the total variability is retained.
* Redefine the data in the basis yielded by the PCA procedure
* Plot the data points in the original and the new coordiantes as a set of scatter plots. Your final figure should have 2 rows of 3 plots each, where the columns show the (0,1), (0,2) and (1,2) proejctions.


In [124]:
import numpy as np
from scipy import linalg as la
from matplotlib import pyplot as plt
from scipy.stats import norm, multivariate_normal
import pandas as pd

#Seeding the random generator
np.random.seed(2009440)

In [125]:
#Using the linalg.eig method
N = 3000

x1 = np.random.normal(0, 1, N)
x2 = x1 + np.random.normal(0, 3, N)
x3 = 2*x1 + x2

dataset = pd.DataFrame(np.array([x1, x2, x3]), index = ['x1', 'x2', 'x3'])

covariance = np.cov(dataset.values)

vals, vecs = la.eig(covariance)
print("eigenvalues {} and eigenvectors {}".format(np.real_if_close(vals), vecs))

eigenvalues [ 2.83470231e+01 -7.37553010e-16  2.01423137e+00] and eigenvectors [[-0.11971957 -0.81649658  0.56480134]
 [-0.57201798 -0.40824829 -0.71142727]
 [-0.81145712  0.40824829  0.41817542]]


In [126]:
#Using SVD

U, spectrum, Vt = la.svd(dataset.values)
U, spectrum**2/(N-1)

# I don't quite understand why the eigenvectors have their two last coordinates flipped.
#I suppose that the last eigenvalue is just the trivial eigenvalue. It corresponds to the eigenvalue (2.67907212e-16) in the last exercise, and it's probably just different from 0 because of how the la.eig and la.svd methods are built.

(array([[-0.11971991,  0.56480127, -0.81649658],
        [-0.57201756, -0.71142761, -0.40824829],
        [-0.81145737,  0.41817493,  0.40824829]]),
 array([2.83470242e+01, 2.01444402e+00, 1.52255190e-31]))

2\. **PCA on a nD dataset**

Start from the dataset you have genereted in the previous exercise and add uncorrelated random noise. Such noise should be represented by other 10 uncorrelated variables normal distributed, with standar deviation much smaller (say, a factor 50) than those used to generate the $x_1$ and $x_2$.

Repeat the PCA procedure and compare the results with what you obtained before

In [127]:
# Setting up the dataset

dataset = dataset.T
dataset['noise1'] = norm.rvs(0, 0.02, N)
dataset['noise2'] = norm.rvs(0, 0.02, N)
dataset['noise3'] = norm.rvs(0, 0.02, N)
dataset['noise4'] = norm.rvs(0, 0.02, N)
dataset['noise5'] = norm.rvs(0, 0.02, N)
dataset['noise6'] = norm.rvs(0, 0.02, N)
dataset['noise7'] = norm.rvs(0, 0.02, N)
dataset['noise8'] = norm.rvs(0, 0.02, N)
dataset['noise9'] = norm.rvs(0, 0.02, N)
dataset['noise10'] = norm.rvs(0, 0.02, N)

dataset

,x1,x2,x3,noise1,noise2,noise3,noise4,noise5,noise6,noise7,noise8,noise9,noise10
0,0.050952,-0.495919,-0.394015,0.003268,0.012137,0.042711,0.015946,0.012253,-0.000374,0.001712,-0.014732,-0.007206,-0.021446
1,0.397304,-3.421277,-2.626668,0.010552,-0.031792,-0.005549,0.024889,0.022918,0.012311,0.014014,-0.005003,0.010906,-0.009512
2,-1.741875,-0.449798,-3.933547,0.009694,-0.016230,0.020789,-0.029677,-0.015939,0.014588,-0.028885,0.007077,0.010783,-0.026327
3,-1.054188,-2.926740,-5.035117,-0.007424,-0.023782,-0.017329,-0.005848,0.014804,0.006550,-0.002450,-0.016918,-0.002513,-0.001043
4,-0.494679,1.410679,0.421322,0.008276,0.001692,-0.009837,0.004536,-0.035035,-0.024687,0.003455,-0.005071,-0.026977,-0.002844
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.286966,0.894269,1.468201,-0.020276,-0.016796,-0.000539,0.005699,-0.030391,-0.019631,0.007496,0.013288,0.017239,0.004366
2996,0.580927,0.559708,1.721561,0.008686,-0.003003,-0.031433,-0.000624,-0.006238,0.006143,-0.027823,-0.005379,-0.000972,-0.015521
2997,-0.068770,-4.169455,-4.306994,0.034579,-0.007913,0.031498,0.018390,-0.009259,0.016227,0.021041,0.008630,0.013091,0.000936
2998,0.163586,-2.037988,-1.710816,-0.006504,0.008919,-0.014474,0.022823,-0.020645,0.001046,-0.004875,0.035243,0.003710,0.018563


In [137]:
# PCA 1 - linalg.eig

covariance = np.cov(dataset.T.values)

vals, vecs = la.eig(covariance)
print("eigenvectors {} and \n eigenvalues {}".format(vecs, np.real_if_close(vals),))

eigenvectors [[-1.19719568e-01 -5.64801005e-01  8.16496581e-01 -1.50240865e-04
  -3.61180824e-04 -7.12812246e-05  1.13895939e-06  3.05118776e-04
  -1.84913209e-04  5.87141248e-05 -1.27666001e-04  2.63188909e-04
   9.27702809e-05]
 [-5.72017974e-01  7.11426866e-01  4.08248290e-01  1.99434016e-04
   4.27848661e-04  1.33718821e-04 -7.49692870e-06 -3.78900732e-04
   2.77526257e-04 -5.19851650e-05  1.95353126e-04 -2.59211789e-04
  -1.28290243e-04]
 [-8.11457109e-01 -4.18175145e-01 -4.08248290e-01 -1.01047714e-04
  -2.94512987e-04 -8.84362849e-06 -5.21900991e-06  2.31336820e-04
  -9.23001615e-05  6.54430847e-05 -5.99788764e-05  2.67166029e-04
   5.72503187e-05]
 [-2.14810139e-05 -4.60536940e-04  4.50080851e-16  2.06301417e-01
   1.00173272e-01 -2.81201727e-01 -1.54220060e-01 -2.68084758e-01
  -2.19080488e-01 -2.09067389e-01  4.62808808e-01 -1.55853679e-01
  -6.65198735e-01]
 [ 6.08731632e-05 -7.07306167e-04 -5.22088035e-15  2.35825685e-01
   1.69695778e-01  3.36397795e-01  3.94208319e-01 -3.

In [134]:
# PCA 2 - linalg.SVD

U, spectrum, Vt = la.svd(dataset.T.values)
U, spectrum**2/(N-1)

(array([[-1.19719905e-01,  5.64800936e-01, -1.04019618e-06,
          6.92403491e-05, -3.04603591e-04,  6.30242171e-05,
         -1.28948121e-04, -8.68013741e-05, -2.61571399e-04,
         -1.85394885e-04,  3.54774342e-04,  1.62008461e-04,
          8.16496581e-01],
        [-5.72017549e-01, -7.11427210e-01,  7.45664042e-06,
         -1.31341841e-04,  3.78178811e-04, -5.83861072e-05,
          1.97916187e-04,  1.19340961e-04,  2.56996705e-04,
          2.78129382e-04, -4.20186132e-04, -2.13271085e-04,
          4.08248290e-01],
        [-8.11457359e-01,  4.18174662e-01,  5.37624805e-06,
          7.13885747e-06, -2.31028371e-04,  6.76623269e-05,
         -5.99800551e-05, -5.42617870e-05, -2.66146093e-04,
         -9.26603885e-05,  2.89362552e-04,  1.10745838e-04,
         -4.08248290e-01],
        [-2.14981946e-05,  4.63753502e-04,  1.54088100e-01,
          2.79940080e-01,  2.55146683e-01, -2.30721880e-01,
          4.81999389e-01,  6.50151675e-01,  1.44883595e-01,
         -2.1774609

3 \. **Looking at an oscillating spring** (optional)

Imagine you have $n$ cameras looking at a spring oscillating along the $x$ axis. Each  camera record the motion of the spring looking at it along a given direction defined by the pair $(\theta_i, \phi_i)$, the angles in spherical coordinates. 

Start from the simulation of the records (say ${\cal O}(1000)$) of the spring's motion along the x axis, assuming a little random noise affects the measurements along the $y$. Rotate such dataset to emulate the records of each camera.

Perform a Principal Component Analysis on the thus obtained dataset, aiming at finding the only one coordinate that really matters.


4\. **PCA on the MAGIC dataset** (optional)

Perform a PCA on the magic04.data dataset

In [ ]:
# get the dataset and its description on the proper data directory
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.data -P ~/data/
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.names -P ~/data/ 